In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import onezone.data_tables as DT
from galaxy_analysis.utilities import convert_abundances as convert
import onezone.imf as imf

/home/aemerick/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/aemerick/code/galaxy_analysis/particle_analysis/IMF.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/aemerick/anaconda2/lib/python2.7/sit

/home/aemerick/code/galaxy_analysis/particle_analysis/sn_rate.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", li

/home/aemerick/code/galaxy_analysis/plot/plot_styles.py:7: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, i

In [2]:
CCSN = DT.StellarYieldsTable('SNII')
wind = DT.StellarYieldsTable('wind')


In [3]:
convert.get_yield_ratio?

In [4]:
def interpolate_many(vals_list, fields, table):
    return np.array([table.interpolate( [x,y], fields)[0] for x,y in vals_list])

def get_abundance_ratio(vals_list, element1, element2, yield_type):
    
    #mass1 = interpolate_many(vals_list, element1, table)
    #mass2 = interpolate_many(vals_list, element2, table)
    
    aratios = np.array( [convert.get_yield_ratio(element1,element2,x,y,yield_type) for x,y in vals_list]  )
    return aratios

In [ ]:
CCSN.interpolate( [10.0,0.001], 'O')

In [ ]:
get_abundance_ratio(  [[10.0,0.001], [15.0,0.001]], 'O','Fe','SNII')

In [ ]:
mass_values = np.linspace(1.0,100.0,128.0)
z_values    = np.ones(128)*1.0E-4
#np.linspace(1.0E-4, 0.01,128.0)

mass_centers = 0.5*(mass_values[1:] +  mass_values[:-1])
z_centers    = 0.5*(z_values[1:] + z_values[:-1])

plot_mmesh, plot_zmesh = np.meshgrid(mass_values, z_values)

mmesh, zmesh = np.meshgrid(  mass_centers, z_centers)

mflat, zflat = mmesh.flatten(), zmesh.flatten()


vals = zip(mflat, zflat)

SNII_aratios = get_abundance_ratio( vals, 'Ba','Fe','SNII').reshape(127,127)
wind_aratios = get_abundance_ratio( vals, 'Ba','Fe','wind').reshape(127,127)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(8,8)

p = ax.pcolormesh(plot_mmesh, plot_zmesh, SNII_aratios)
ax.set_xlim(1.0,100.0)
ax.set_ylim(0.0001,0.01)
plt.colorbar(p)
ax.colorbar

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(8,8)

SNII_aratios[0][(mass_centers < 8.0) + (mass_centers>25.0)] = np.nan
ax.plot(mass_centers, SNII_aratios[0], label = 'SNII')
ax.set_xlim(1.0,25.0)

select = mass_centers > 8.0
ax.plot(mass_centers[select], wind_aratios[0][select], label = 'Winds')

select = mass_centers < 8.0
ax.plot(mass_centers[select], wind_aratios[0][select], label = 'AGB Winds')

ax.legend(loc='best')

In [ ]:
onez

In [17]:


def imf_weighted_yields(IMF, yield_table = wind):
    """
    IMF is some funtion that can be evaluated to give an arbitrarily
    normalized IMF weighted at some mass "m"
    """
    
    z_values = 1.0*yield_table.x['metallicity']
    y_names  = yield_table.y_names()
    
    m_values   = 1.0* yield_table.x['mass']
    
    m_values[-1] = 0.999*m_values[-1]
    z_values[-1] = 0.999*z_values[-1]
    
    print np.size(m_values),np.size(z_values)
    
    mmesh, zmesh = np.meshgrid(m_values, z_values)
    
    masses, z = mmesh.flatten(), zmesh.flatten()
    
    vals = zip(masses,z)
    
    result = {}
    for x in y_names:
        temp = interpolate_many(vals, x, yield_table) * IMF(masses)
        
        result[x] = temp.reshape(np.size(z_values),np.size(m_values))
        
        print x, result[x]
    
    
    return result
    
    
    


In [14]:
salpeter = imf.salpeter(M_min = 1.0, M_max = 100.0)
interpolate_many([(1.0,0.0001),(22.5,0.01802)], 'O', CCSN)

array([0.        , 1.15599939])

In [18]:
salpeter = imf.salpeter(M_min = 1.0, M_max = 100.0)
f_imf = lambda x : salpeter.imf(x)

result = imf_weighted_yields(f_imf, yield_table = CCSN)

12 5
m_tot [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
m_metal [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
H [[0.         0.         0.         0.         0.         0.
  0.         0.         0.1905894  0.16859645 0.13952822 0.11995776]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.19002545 0.16691565 0.13695321 0.11683793]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.1637798  0.15247387 0.12353746 0.08136222]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.17300286 0.14007367 0.10613707 0.07480007]
 [0.         0.         0.         0.         0.         0.
  0.       

  1.41480404e-03 1.09312837e-03 9.85203838e-04 4.47685657e-04]]
Mg [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Al [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Si [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.90183945e-03 1.15234061e-03 5.25434133e-04 2.83679636e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.72969050e-03 1.04631022e-03 2.10784902e-03 2.41047111e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.18137748e-03 1.82402914e-03 3.591

  0.         0.         0.00267099 0.00380388 0.00337155 0.00264097]]
Mn [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Fe [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  9.60310761e-10 1.16243599e-09 1.63239334e-09 3.33984340e-08]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.96811437e-08 1.03867048e-08 4.88458305e-09 5.72604622e-09]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.08972865e-07 6.81709477e-08 1.53940276e-07 3.09453318e-08]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.81286186e-07 1.82638266e-07 1.87433468e

  1.35058815e-03 5.18162660e-03 1.12196191e-03 2.17927838e-04]]
Y [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  5.14302949e-12 1.04175148e-10 8.73198249e-12 4.95678658e-12]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.58627998e-10 2.21944083e-09 3.53909360e-09 1.58068759e-09]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.32329610e-09 6.11592850e-07 4.51032591e-08 2.80023914e-08]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.47548137e-09 5.91559562e-09 8.67127741e-09 3.82325694e-09]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.51976381e-09 1.27562158e-08 2.29168241e-08 6.98455490e-09]

  4.47453533e-10 3.53605646e-10 2.88633487e-10 1.38114378e-10]]
Xe [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Cs [[0.         0.         0.         0.         0.         0.
  0.         0.         0.00428157 0.00425319 0.0034289  0.00264381]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.00419134 0.00397201 0.00389294 0.00274664]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.00354275 0.00427418 0.00258186 0.00285989]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.00311282 0.00406806 0.00402061 0.00290574]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.00267099 0.00380388 0.00337155 0.00264097]]
Ba [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.0000

 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Yb [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  5.14302949e-12 1.04175148e-10 8.73198249e-12 4.95678658e-12]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.58627998e-10 2.21944083e-09 3.53909360e-09 1.58068759e-09]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.32329610e-09 6.11592850e-07 4.51032591e-08 2.80023914e-08]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.47548137e-09 5.91559562e-09 8.67127741e-09 3.82325694e-09]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.51976381e-09 1.27562158e-08 2.29168241e-08 6.98455490e-09]]
Lu [[0. 0. 0. 0. 0. 0

  8.53921112e-10 6.04889006e-10 4.45774838e-10 1.40234920e-10]]


In [19]:
result['O']

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00987627, 0.03038689,
        0.0358    , 0.00925599],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.01208031, 0.0296623 ,
        0.03278268, 0.01029446],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.0065624 , 0.02225137,
        0.02527081, 0.01159507],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00416942, 0.02281369,
        0.02302238, 0.01105867],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00420957, 0.02317754,
        0.02634881, 0.01100895]])